## Importing Dependencies

In [1]:
import os
import numpy as np

import cv2
import matplotlib.pylab as plt

import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Loading Datasets

In [3]:
train_dir = 'imgdata_v2/split_dataset/train_aug/'
test_dir = 'imgdata_v2/split_dataset/test/'
val_dir = 'imgdata_v2/split_dataset/val/'

In [4]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(256, 256))
val_dataset = tf.keras.utils.image_dataset_from_directory(val_dir, image_size=(256, 256))

Found 7051 files belonging to 2 classes.
Found 415 files belonging to 2 classes.


## Scaling Dataset

In [5]:
train_dataset.as_numpy_iterator().next()[0][0].max()

255.0

In [6]:
train_data = train_dataset.map(lambda x, y: (x/255, y))

In [7]:
train_data.as_numpy_iterator().next()[0][0].max()

0.75980395

In [8]:
val_data = val_dataset.map(lambda x, y: (x/255, y))

## CNN Model Architecture

In [9]:
import tensorflow as tf
from keras import Sequential,Model,models
from keras.layers import Dense,Dropout,Flatten,BatchNormalization, Conv2D, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2 , preprocess_input

In [10]:
inception_arch=InceptionResNetV2(input_shape=[256, 256] + [3],weights="imagenet",include_top=False)

In [11]:
for layer in inception_arch.layers:
    layer.trainaable = False

In [12]:
model = Sequential()
model.add(inception_arch)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='linear'))

In [13]:
model.compile(optimizer="adam",loss="mse",metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 6, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 55296)             0         
                                                                 
 dense (Dense)               (None, 256)               14156032  
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 68,493,025
Trainable params: 68,432,481
Non-trainable params: 60,544
_________________________________________________________________


In [14]:
logdir = 'LogDir'

try:
    os.mkdir(logdir)
except Exception as e:
    pass    

tfb_callback = tf.keras.callbacks.TensorBoard(logdir)

In [15]:
model_fit = model.fit(train_data, 
                       epochs=5, 
                       shuffle=True, 
                       validation_data=val_data,
                       callbacks=[tfb_callback])

Epoch 1/5
221/221 [==============================] - 124s 452ms/step - loss: 51.1960 - accuracy: 0.4903 - val_loss: 1.5560 - val_accuracy: 0.4843
Epoch 2/5
221/221 [==============================] - 99s 449ms/step - loss: 0.4950 - accuracy: 0.4889 - val_loss: 0.5112 - val_accuracy: 0.4843
Epoch 3/5
221/221 [==============================] - 101s 456ms/step - loss: 0.4719 - accuracy: 0.4889 - val_loss: 0.4714 - val_accuracy: 0.4843
Epoch 4/5
 67/221 [========>.....................] - ETA: 1:10 - loss: 0.4483 - accuracy: 0.4963

KeyboardInterrupt: 

## Model Performance 

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,5))
axs[0].plot(model_fit.history['loss'], 
            color='red', 
            label='Loss')
axs[0].plot(model_fit.history['val_loss'], 
            color='orange', 
            label='Validation Loss')
axs[0].legend(loc='upper right')
axs[0].set_xlabel('Epochs', fontsize=15)
axs[0].set_ylabel('Loss', fontsize=15)


axs[1].plot(model_fit.history['accuracy'], 
             color='green', 
             label='Accuracy')
axs[1].plot(model_fit.history['val_accuracy'], 
             color='teal', 
             label='Validation Accuracy')
axs[1].legend(loc='upper left')
axs[1].set_xlabel('Epochs', fontsize=15)
axs[1].set_ylabel('Accuracy', fontsize=15)

fig.suptitle('InceptionResNet_V2 Model Performance Curves', fontsize=20)

## Exporting Model

In [ ]:
from keras.models import save_model, load_model

model.save(os.path.join('models_Reboot/','Inception_Re.h5'))

In [ ]:
Model = load_model(os.path.join('models_Reboot/','Inception_Re.h5'))

In [ ]:
Model.summary()

## Evaluating Model

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(256, 256))

In [ ]:
test_data = test_dataset.map(lambda x, y : (x/255, y))

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
acc = BinaryAccuracy()
rec = Recall()

In [ ]:
for batch in test_data.as_numpy_iterator():
    X, y = batch
    y_pred = model.predict(X)
    
    acc.update_state(y, y_pred)
    pre.update_state(y, y_pred)
    rec.update_state(y, y_pred)

In [ ]:
accuracy = acc.result().numpy()
precision = pre.result().numpy()
recall = rec.result().numpy()
f1 = 2 * (precision*recall) / (precision+recall)

print('Accuracy\t:', round(accuracy,5))
print('Precision\t:', round(precision,5))
print('Recall\t\t:', round(recall,5))
print('F1\t\t:', round(f1,5))

In [ ]:
import seaborn as sns

Y=[]
Y_pred=[]
for batch in test_data.as_numpy_iterator():
    X, y = batch
    y_pred = model.predict(X)
    y_pred = (y_pred > 0.5)
    Y.extend(y)
    Y_pred.extend(y_pred)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y, Y_pred)

plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='copper', cbar=False,
            xticklabels=['Cataract', 'Normal'], yticklabels=['Cataract', 'Normal'])
plt.xlabel('PREDICTED', fontsize=12)
plt.ylabel('TRUE', fontsize=12)
plt.title('CNN Model Confusion Matrix', fontsize=15)
plt.show()